<a href="https://colab.research.google.com/github/mirpouya/TensorFlow-Tutorial/blob/main/Convolutional_Neural_Net_Malaria_Detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [3]:
dataset, dataset_info = tfds.load("malaria", with_info=True, as_supervised=True, shuffle_files=True, split=["train"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteT8FYQI/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [4]:
dataset

[<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>]

In [5]:
type(dataset)

list

In [6]:
len(dataset)

1

In [7]:
len(dataset[0])

27558

In [8]:
print(dataset[0])

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [9]:
dataset_info

tfds.core.DatasetInfo(
    name='malaria',
    full_name='malaria/1.0.0',
    description="""
    The Malaria dataset contains a total of 27,558 cell images with equal instances
    of parasitized and uninfected cells from the thin blood smear slide images of
    segmented cells.
    """,
    homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
    data_path=PosixGPath('/tmp/tmpzttozo5ptfds'),
    file_format=tfrecord,
    download_size=337.08 MiB,
    dataset_size=317.62 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=27558, num_shards=4>,
    },
    citation="""@article{rajaraman2018pre,
      title={Pre-trained convolutional neural networks as feature extractors toward
      improved malaria parasite detection in thin blood smear images},

Models hyper parameters

In [10]:
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

Splitting dataset

In [11]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  size = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO * size))
  val_dataset = dataset.skip(int(TRAIN_RATIO * size)).take(int(VAL_RATIO * size))
  test_dataset = dataset.skip(int((TRAIN_RATIO + VAL_RATIO) * size))

  return train_dataset, val_dataset, test_dataset

splitting train, validation, and test

In [12]:
train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

Resize and rescaling images

In [13]:
IM_SIZE = 224
def resize_rescale(image, label, img_size = IM_SIZE):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE)) / 255.0, label

resize and rescalling train, validation, and test

In [14]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

In [15]:
train_dataset, val_dataset, test_dataset

(<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [16]:
type(train_dataset)

tensorflow.python.data.ops.map_op._MapDataset

In [17]:
train_dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [18]:
type(train_dataset.take(1))

tensorflow.python.data.ops.take_op._TakeDataset

In [19]:
for image, label in train_dataset.take(1):
  print(image, label)

tf.Tensor(
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]], shape=(224, 224, 3), dtype=float32) tf.Tensor(0, shape=(), dtype=int64)


shuffling train, validation, and test sets

In [20]:
BATCH_SIZE = 32

train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(1).prefetch(tf.data.AUTOTUNE)

In [21]:
train_dataset, val_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

Model creation

In [22]:
from tensorflow.keras.layers import InputLayer, Conv2D, Dense, BatchNormalization, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [23]:
from keras.api._v2.keras import activations
model = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)),

    Conv2D(filters= 6, kernel_size= 5, strides= 1, padding= "valid",activation= "relu"),
    BatchNormalization(),
    MaxPool2D(pool_size= 2, strides= 2),

    Conv2D(filters=16, kernel_size=5, strides=1, padding="valid", activation="relu"),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation="relu"),
    BatchNormalization(),

    Dense(10, activation="relu"),
    BatchNormalization(),

    Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 6)       456       
                                                                 
 batch_normalization (Batch  (None, 220, 220, 6)       24        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 6)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_1 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                        

Compiling the model: setting optimizar, loss, and metrics

In [24]:
model.compile(optimizer = Adam(learning_rate=0.08),
              loss=BinaryCrossentropy(),
              metrics="accuracy")

fitting the model

In [24]:
history = model.fit(train_dataset, validation_data = val_dataset, epochs=3, verbose=1)

Epoch 1/3
603/603 [==============================] - 56s 71ms/step - loss: 0.3833 - accuracy: 0.8415 - val_loss: 0.4592 - val_accuracy: 0.8737
Epoch 2/3
603/603 [==============================] - 47s 78ms/step - loss: 0.2408 - accuracy: 0.9259 - val_loss: 0.5965 - val_accuracy: 0.7574
Epoch 3/3
603/603 [==============================] - 47s 78ms/step - loss: 0.2129 - accuracy: 0.9337 - val_loss: 0.2968 - val_accuracy: 0.9196


In [25]:
test_dataset, val_dataset, train_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

Evaluating the model

In [26]:
model.evaluate(test_dataset)

2756/2756 [==============================] - 21s 4ms/step - loss: 0.3022 - accuracy: 0.9180


[0.3021606504917145, 0.9179971218109131]

predicting

In [27]:
test_dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [28]:
model.predict(test_dataset.take(1))

1/1 [==============================] - 11s 11s/step


array([[0.9959925]], dtype=float32)

In [29]:
model.predict(test_dataset.take(1))[0]

1/1 [==============================] - 11s 11s/step


array([0.19300343], dtype=float32)

In [30]:
model.predict(test_dataset.take(1))[0][0]

1/1 [==============================] - 11s 11s/step


0.9980028

Saving the model

In [31]:
model.save("model")

Saving the model in Google Drive

In [32]:
from google.colab import drive

In [33]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [34]:
!cp -r /content/model/ /content/drieve/MyDrive/models

cp: cannot create directory '/content/drieve/MyDrive/models': No such file or directory


<h2> <b> Functional API </b> </h2>

In [25]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

In [26]:
func_api_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input Image")

x = Conv2D(filters=6, kernel_size=5, strides=1, padding="valid", activation="relu")(func_api_input)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=2, strides=2)(x)

x = Conv2D(filters=16, kernel_size=5, strides=1, padding="valid", activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=2, strides=2)(x)

x = Flatten()(x)

x = Dense(100, activation="relu")(x)
x = BatchNormalization()(x)
x = Dense(10, activation="relu")(x)
x = BatchNormalization()(x)

func_api_output = Dense(1, activation="sigmoid")(x)

Lenet_model = Model(func_api_input, func_api_output, name="Lenet_model")
Lenet_model.summary()

Model: "Lenet_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_4 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_5 (Bat  (None, 106, 106, 16)      

In [27]:
Lenet_model.compile(
    optimizer = Adam(learning_rate = 0.05),
    loss = BinaryCrossentropy(),
    metrics = "accuracy"
)

In [38]:
history = Lenet_model.fit(train_dataset, validation_data = val_dataset, epochs=3, verbose=1)

Epoch 1/3
603/603 [==============================] - 54s 82ms/step - loss: 0.5845 - accuracy: 0.6872 - val_loss: 0.7794 - val_accuracy: 0.4946
Epoch 2/3
603/603 [==============================] - 47s 77ms/step - loss: 0.2543 - accuracy: 0.9136 - val_loss: 0.6808 - val_accuracy: 0.5028
Epoch 3/3
603/603 [==============================] - 44s 72ms/step - loss: 0.1979 - accuracy: 0.9341 - val_loss: 0.1717 - val_accuracy: 0.9428


Aother way of arranging the same model

In [28]:
feature_extrac_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input Image")

x = Conv2D(filters=6, kernel_size=5, strides=1, padding="valid", activation="relu")(feature_extrac_input)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=2, strides=2)(x)

x = Conv2D(filters=16, kernel_size=5, strides=1, padding="valid", activation="relu")(x)
x = BatchNormalization()(x)

feature_extract_output = MaxPool2D(pool_size=2, strides=2)(x)

feature_extract_model = Model(feature_extrac_input, feature_extract_output, name="feature_extractor_model")

feature_extract_model.summary()

Model: "feature_extractor_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_8 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_9 (Bat  (None, 106, 10

In [29]:
lenet_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input Image")

features = feature_extract_model(lenet_input)

x = Flatten()(features)

x = Dense(100, activation="relu")(x)
x = BatchNormalization()(x)
x = Dense(10, activation="relu")(x)
x = BatchNormalization()(x)
lenet_output = Dense(1, activation="sigmoid")(x)

Lenet_model_1 = Model(lenet_input, lenet_output, name="Lenet_model_1")

Lenet_model_1.summary()

Model: "Lenet_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 feature_extractor_model (F  (None, 53, 53, 16)        2960      
 unctional)                                                      
                                                                 
 flatten_2 (Flatten)         (None, 44944)             0         
                                                                 
 dense_6 (Dense)             (None, 100)               4494500   
                                                                 
 batch_normalization_10 (Ba  (None, 100)               400       
 tchNormalization)                                               
                                                                 
 dense_7 (Dense)             (None, 10)              

<h2> <b> Model Subclassing </b> </h2>

In [30]:
from tensorflow.keras.layers import Layer

In [31]:
from keras.src.utils.feature_space import Feature
class FeatureExtractor(Layer):
  def __init__(self, filters=6, kernel_size=3, strides=1, padding="valid", activation="relu", pool_size=2):
    super(FeatureExtractor, self).__init__()

    self.conv2d_1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation)
    self.batchnorm_1 = BatchNormalization()
    self.maxpool_1 = MaxPool2D(pool_size = pool_size, strides = 2*strides)

    self.conv2d_2 = Conv2D(filters = 2*filters, kernel_size= kernel_size, strides=strides, padding=padding, activation=activation)
    self.batchnorm_2 = BatchNormalization()
    self.maxpool_2 = MaxPool2D(pool_size = pool_size, strides= 2*strides)

  def call(self, x, training):
    x = self.conv2d_1(x)
    x = self.batchnorm_1(x)
    x = self.maxpool_1(x)

    x = self.conv2d_2(x)
    x = self.batchnorm_2(x)
    x = self.maxpool_2(x)

    return x

In [32]:
feature_extract_subclassed = FeatureExtractor(filters=8, kernel_size=3, strides=1, padding="valid", activation="relu", pool_size=2)

In [33]:
lenet_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input Image")

features = feature_extract_subclassed(lenet_input)

x = Flatten()(features)

x = Dense(100, activation="relu")(x)
x = BatchNormalization()(x)
x = Dense(10, activation="relu")(x)
x = BatchNormalization()(x)
lenet_output = Dense(1, activation="sigmoid")(x)

Lenet_model_1 = Model(lenet_input, lenet_output, name="Lenet_model_1")

Lenet_model_1.summary()

Model: "Lenet_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 feature_extractor (Feature  (None, 54, 54, 16)        1488      
 Extractor)                                                      
                                                                 
 flatten_3 (Flatten)         (None, 46656)             0         
                                                                 
 dense_9 (Dense)             (None, 100)               4665700   
                                                                 
 batch_normalization_14 (Ba  (None, 100)               400       
 tchNormalization)                                               
                                                                 
 dense_10 (Dense)            (None, 10)              

In [34]:
class LenetModel(Model):
  def __init__(self):
    super(LenetModel, self).__init__()
    self.feature_extractor = FeatureExtractor(filters=8, kernel_size=3, strides=1, padding="valid", activation="relu", pool_size=2)

    self.flatten = Flatten()
    self.dense_1 = Dense(100, activation="relu")
    self.dense_2 = Dense(10, activation="relu")
    self.dense_3 = Dense(1, activation="sigmoid")
    self.batchnorm_1 = BatchNormalization()
    self.batchnorm_2 = BatchNormalization()

  def call(self, x, training):
    x = self.feature_extractor(x)
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.batchnorm_1(x)
    x = self.dense_2(x)
    x = self.batchnorm_2(x)
    outcome = self.dense_3(x)

In [35]:
lenet_model_subclass = LenetModel()

In [36]:
lenet_model_subclass(tf.zeros([1, 224, 224, 3]))

In [37]:
lenet_model_subclass.summary()

Model: "lenet_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_1 (Featu  multiple                  1488      
 reExtractor)                                                    
                                                                 
 flatten_4 (Flatten)         multiple                  0         
                                                                 
 dense_12 (Dense)            multiple                  4665700   
                                                                 
 dense_13 (Dense)            multiple                  1010      
                                                                 
 dense_14 (Dense)            multiple                  11        
                                                                 
 batch_normalization_18 (Ba  multiple                  400       
 tchNormalization)                                     

<h2> <b> Custom Layers </b> </h2>

In [50]:
class NeuralDense(Layer):
  def __init__(self, output_units):
    super(NeuralDense, self).__init__()
    self.output_units = output_units

  def build(self, input_features_shape):
    # 'add_weight()' is inherited from 'Layer'
    self.w = self.add_weight(shape = (input_features_shape[-1], self.output_units), trainable=True, initializer="random_normal")
    self.b = self.add_weight(shape = (self.output_units,), trainable=True, initializer="random_normal")

  def call(self, input_features):
    return tf.matmul(input_features, self) + self.b

In [57]:
class NeuralearnDense(Layer):
  def __init__(self, output_units, activation):
    super(NeuralearnDense, self).__init__()
    self.output_units = output_units
    self.activation = activation

  def build(self, input_features_shape):
    self.w = self.add_weight(shape = (input_features_shape[-1], self.output_units), initializer = "random_normal", trainable = True)
    self.b = self.add_weight(shape = (self.output_units,), initializer = "random_normal", trainable = True)

  def call(self, input_features):

    pre_output = tf.matmul(input_features, self.w) + self.b

    if(self.activation == "relu"):
      return tf.nn.relu(pre_output)

    elif(self.activation == "sigmoid"):
      return tf.math.sigmoid(pre_output)

    else:
      return pre_output

In [58]:
IM_SIZE = 224
lenet_custom_model = tf.keras.Sequential([
                             InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)),

                             Conv2D(filters = 6, kernel_size = 3, strides=1, padding='valid', activation = 'relu'),
                             BatchNormalization(),
                             MaxPool2D (pool_size = 2, strides= 2),

                             Conv2D(filters = 16, kernel_size = 3, strides=1, padding='valid', activation = 'relu'),
                             BatchNormalization(),
                             MaxPool2D (pool_size = 2, strides= 2),

                             Flatten(),

                             NeuralearnDense(100, activation = "relu"),
                             BatchNormalization(),

                             NeuralearnDense(10, activation = "relu"),
                             BatchNormalization(),

                             NeuralearnDense(1, activation = "sigmoid"),

])
lenet_custom_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 222, 222, 6)       168       
                                                                 
 batch_normalization_41 (Ba  (None, 222, 222, 6)       24        
 tchNormalization)                                               
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 111, 111, 6)       0         
 ng2D)                                                           
                                                                 
 conv2d_23 (Conv2D)          (None, 109, 109, 16)      880       
                                                                 
 batch_normalization_42 (Ba  (None, 109, 109, 16)      64        
 tchNormalization)                                               
                                                      